In [ ]:
from IPython.display import FileLink
import os
import numpy as np
import pandas as pd
import PIL
from pathlib import Path
import shutil

In [ ]:
!git clone https://github.com/WongKinYiu/yolov7.git
!ls yolov7

In [ ]:
!pip install -r yolov7/requirements.txt

# Data

The input dataset needs to be organized according to the following structure:

```
images/
  train/
  valid/
labels/
  train/
  valid/
```

As an example, we'll use the Kaggle "Cyclist Dataset for Object Detection" (https://www.kaggle.com/datasets/semiemptyglass/cyclist-dataset) uploaded by Kaggle user "SEMIEMPTYGLASS".
Original authors: X. Li, F. Flohr, Y. Yang, H. Xiong, M. Braun, S. Pan, K. Li and D. M. Gavrila.

In [ ]:
orig_ds_root = Path('/kaggle/input/cyclist-dataset/data_tsinghua')
ds_root = Path('/kaggle/working/cyclist-dataset')
!mkdir $ds_root

In [ ]:
all_img_paths = sorted((orig_ds_root/'images').glob('*'))
all_label_paths = sorted((orig_ds_root/'labels').glob('*'))
assert len(all_img_paths) == len(all_label_paths)

In [ ]:
valid_pct = 0.2
split_idx = int(len(all_img_paths) * (1 - valid_pct))
train_img_paths = all_img_paths[:split_idx]
val_img_paths = all_img_paths[split_idx:]
train_label_paths = all_label_paths[:split_idx]
val_label_paths = all_label_paths[split_idx:]
!mkdir {ds_root/'images'}
!mkdir {ds_root/'labels'}

for split, label_paths, img_paths in zip(
    ('train', 'valid'), (train_label_paths, val_label_paths), (train_img_paths, val_img_paths), 
):
    !mkdir {ds_root/'images'/split}
    !mkdir {ds_root/'labels'/split}
    
    i = 0
    for p in img_paths:
        # Faster than !cp $p {ds_root/images'/split}
        shutil.copy(p, ds_root/'images'/split)
        i += 1
        if i%1000 == 0:
            print('Done ', i)
    for p in label_paths:
        shutil.copy(p, ds_root/'labels'/split)

In [ ]:
def test_ds_reorganization(orig_ds_root, ds_root, valid_pct):
    for artifact in ('images', 'labels'):
        actual_count = len(list((orig_ds_root/artifact).glob('*')))
        expected_train_count = int(actual_count * (1 - valid_pct))
        expected_valid_count = actual_count - expected_train_count

        train_items = os.listdir(ds_root/artifact/'train')
        valid_items = os.listdir(ds_root/artifact/'valid')
        train_valid_are_disjoint = 0 == len(set(train_items).intersection(set(valid_items)))
        
        assert len(train_items) == expected_train_count
        assert len(valid_items) == expected_valid_count
        assert train_valid_are_disjoint


test_ds_reorganization(orig_ds_root, ds_root, valid_pct)

# YOLO config

Create a metadata file about the input dataset with the format expected by YOLO

In [ ]:
##!! Writefile is not dynamic (doesn't evaluate its content)
# %%writefile yolov7/data/cyclist-dataset.yaml
# train: {ds_root/'images/train'}
# val: {ds_root/'images/valid'}
# #test:

# nc: 1  # number of classes
# names: ['cyclist']  # class names

In [ ]:
data_yaml_content = f"""train: {ds_root/'images/train'}
val: {ds_root/'images/valid'}
#test:

nc: 1  # number of classes
names: ['cyclist']  # class names
"""    
with open('/kaggle/working/yolov7/data/cyclist-dataset.yaml', 'w') as f:
    print(data_yaml_content, file=f)

In [ ]:
!cat data/cyclist-dataset.yaml

Create a copy of yolov7/cfg/training/yolov7-tiny.yaml and change nc (number of classes) to 1:

In [ ]:
%%writefile yolov7/cfg/training/yolov7-tiny-single-class.yaml
# parameters
nc: 1  # number of classes
depth_multiple: 1.0  # model depth multiple
width_multiple: 1.0  # layer channel multiple

# anchors
anchors:
  - [10,13, 16,30, 33,23]  # P3/8
  - [30,61, 62,45, 59,119]  # P4/16
  - [116,90, 156,198, 373,326]  # P5/32

# yolov7-tiny backbone
backbone:
  # [from, number, module, args] c2, k=1, s=1, p=None, g=1, act=True
  [[-1, 1, Conv, [32, 3, 2, None, 1, nn.LeakyReLU(0.1)]],  # 0-P1/2  
  
   [-1, 1, Conv, [64, 3, 2, None, 1, nn.LeakyReLU(0.1)]],  # 1-P2/4    
   
   [-1, 1, Conv, [32, 1, 1, None, 1, nn.LeakyReLU(0.1)]],
   [-2, 1, Conv, [32, 1, 1, None, 1, nn.LeakyReLU(0.1)]],
   [-1, 1, Conv, [32, 3, 1, None, 1, nn.LeakyReLU(0.1)]],
   [-1, 1, Conv, [32, 3, 1, None, 1, nn.LeakyReLU(0.1)]],
   [[-1, -2, -3, -4], 1, Concat, [1]],
   [-1, 1, Conv, [64, 1, 1, None, 1, nn.LeakyReLU(0.1)]],  # 7
   
   [-1, 1, MP, []],  # 8-P3/8
   [-1, 1, Conv, [64, 1, 1, None, 1, nn.LeakyReLU(0.1)]],
   [-2, 1, Conv, [64, 1, 1, None, 1, nn.LeakyReLU(0.1)]],
   [-1, 1, Conv, [64, 3, 1, None, 1, nn.LeakyReLU(0.1)]],
   [-1, 1, Conv, [64, 3, 1, None, 1, nn.LeakyReLU(0.1)]],
   [[-1, -2, -3, -4], 1, Concat, [1]],
   [-1, 1, Conv, [128, 1, 1, None, 1, nn.LeakyReLU(0.1)]],  # 14
   
   [-1, 1, MP, []],  # 15-P4/16
   [-1, 1, Conv, [128, 1, 1, None, 1, nn.LeakyReLU(0.1)]],
   [-2, 1, Conv, [128, 1, 1, None, 1, nn.LeakyReLU(0.1)]],
   [-1, 1, Conv, [128, 3, 1, None, 1, nn.LeakyReLU(0.1)]],
   [-1, 1, Conv, [128, 3, 1, None, 1, nn.LeakyReLU(0.1)]],
   [[-1, -2, -3, -4], 1, Concat, [1]],
   [-1, 1, Conv, [256, 1, 1, None, 1, nn.LeakyReLU(0.1)]],  # 21
   
   [-1, 1, MP, []],  # 22-P5/32
   [-1, 1, Conv, [256, 1, 1, None, 1, nn.LeakyReLU(0.1)]],
   [-2, 1, Conv, [256, 1, 1, None, 1, nn.LeakyReLU(0.1)]],
   [-1, 1, Conv, [256, 3, 1, None, 1, nn.LeakyReLU(0.1)]],
   [-1, 1, Conv, [256, 3, 1, None, 1, nn.LeakyReLU(0.1)]],
   [[-1, -2, -3, -4], 1, Concat, [1]],
   [-1, 1, Conv, [512, 1, 1, None, 1, nn.LeakyReLU(0.1)]],  # 28
  ]

# yolov7-tiny head
head:
  [[-1, 1, Conv, [256, 1, 1, None, 1, nn.LeakyReLU(0.1)]],
   [-2, 1, Conv, [256, 1, 1, None, 1, nn.LeakyReLU(0.1)]],
   [-1, 1, SP, [5]],
   [-2, 1, SP, [9]],
   [-3, 1, SP, [13]],
   [[-1, -2, -3, -4], 1, Concat, [1]],
   [-1, 1, Conv, [256, 1, 1, None, 1, nn.LeakyReLU(0.1)]],
   [[-1, -7], 1, Concat, [1]],
   [-1, 1, Conv, [256, 1, 1, None, 1, nn.LeakyReLU(0.1)]],  # 37
  
   [-1, 1, Conv, [128, 1, 1, None, 1, nn.LeakyReLU(0.1)]],
   [-1, 1, nn.Upsample, [None, 2, 'nearest']],
   [21, 1, Conv, [128, 1, 1, None, 1, nn.LeakyReLU(0.1)]], # route backbone P4
   [[-1, -2], 1, Concat, [1]],
   
   [-1, 1, Conv, [64, 1, 1, None, 1, nn.LeakyReLU(0.1)]],
   [-2, 1, Conv, [64, 1, 1, None, 1, nn.LeakyReLU(0.1)]],
   [-1, 1, Conv, [64, 3, 1, None, 1, nn.LeakyReLU(0.1)]],
   [-1, 1, Conv, [64, 3, 1, None, 1, nn.LeakyReLU(0.1)]],
   [[-1, -2, -3, -4], 1, Concat, [1]],
   [-1, 1, Conv, [128, 1, 1, None, 1, nn.LeakyReLU(0.1)]],  # 47
  
   [-1, 1, Conv, [64, 1, 1, None, 1, nn.LeakyReLU(0.1)]],
   [-1, 1, nn.Upsample, [None, 2, 'nearest']],
   [14, 1, Conv, [64, 1, 1, None, 1, nn.LeakyReLU(0.1)]], # route backbone P3
   [[-1, -2], 1, Concat, [1]],
   
   [-1, 1, Conv, [32, 1, 1, None, 1, nn.LeakyReLU(0.1)]],
   [-2, 1, Conv, [32, 1, 1, None, 1, nn.LeakyReLU(0.1)]],
   [-1, 1, Conv, [32, 3, 1, None, 1, nn.LeakyReLU(0.1)]],
   [-1, 1, Conv, [32, 3, 1, None, 1, nn.LeakyReLU(0.1)]],
   [[-1, -2, -3, -4], 1, Concat, [1]],
   [-1, 1, Conv, [64, 1, 1, None, 1, nn.LeakyReLU(0.1)]],  # 57
   
   [-1, 1, Conv, [128, 3, 2, None, 1, nn.LeakyReLU(0.1)]],
   [[-1, 47], 1, Concat, [1]],
   
   [-1, 1, Conv, [64, 1, 1, None, 1, nn.LeakyReLU(0.1)]],
   [-2, 1, Conv, [64, 1, 1, None, 1, nn.LeakyReLU(0.1)]],
   [-1, 1, Conv, [64, 3, 1, None, 1, nn.LeakyReLU(0.1)]],
   [-1, 1, Conv, [64, 3, 1, None, 1, nn.LeakyReLU(0.1)]],
   [[-1, -2, -3, -4], 1, Concat, [1]],
   [-1, 1, Conv, [128, 1, 1, None, 1, nn.LeakyReLU(0.1)]],  # 65
   
   [-1, 1, Conv, [256, 3, 2, None, 1, nn.LeakyReLU(0.1)]],
   [[-1, 37], 1, Concat, [1]],
   
   [-1, 1, Conv, [128, 1, 1, None, 1, nn.LeakyReLU(0.1)]],
   [-2, 1, Conv, [128, 1, 1, None, 1, nn.LeakyReLU(0.1)]],
   [-1, 1, Conv, [128, 3, 1, None, 1, nn.LeakyReLU(0.1)]],
   [-1, 1, Conv, [128, 3, 1, None, 1, nn.LeakyReLU(0.1)]],
   [[-1, -2, -3, -4], 1, Concat, [1]],
   [-1, 1, Conv, [256, 1, 1, None, 1, nn.LeakyReLU(0.1)]],  # 73
      
   [57, 1, Conv, [128, 3, 1, None, 1, nn.LeakyReLU(0.1)]],
   [65, 1, Conv, [256, 3, 1, None, 1, nn.LeakyReLU(0.1)]],
   [73, 1, Conv, [512, 3, 1, None, 1, nn.LeakyReLU(0.1)]],

   [[74,75,76], 1, IDetect, [nc, anchors]],   # Detect(P3, P4, P5)
  ]

# Run training

In [ ]:
model_name = 'yolov7_tiny_cyclist'
%cd yolov7

In [ ]:
sizes_dict = {}
i = 0
for img_path in (orig_ds_root/'images').glob('*'):
    sz = PIL.Image.open(img_path).size
    sizes_dict[sz] = sizes_dict.get(sz, 0) + 1
    i += 1
    if i % 2000 == 0:
        print('Done ', i)
        #break
    
sizes_dict

You need to either disable wandb or login before training; otherwise, training won't proceed.

In [ ]:
!wandb disabled

To train with a single GPU:

In [ ]:
!python train.py --epochs 50 --workers 4 --device 0 --batch-size 32 \
--data data/cyclist-dataset.yaml --img 640 640 --cfg cfg/training/yolov7-tiny-single-class.yaml \
--weights 'yolov7-tiny.pt' --name $model_name --hyp data/hyp.scratch.tiny.yaml

To train with multiple GPUs (this example assumes 2 GPUs):

In [ ]:
!python -m torch.distributed.launch --nproc_per_node 1 --master_port 9527 train.py --epochs 50 --workers 4 \
--device 0,1 --sync-bn --batch-size 128 --data data/cyclist-dataset.yaml --img 640 640 \
--cfg cfg/training/yolov7-tiny-single-class.yaml --weights 'yolov7-tiny.pt' --name $model_name \
--hyp data/hyp.scratch.tiny.yaml

# Export

Export to TorchScript:

In [ ]:
!python export.py --weights runs/train/{model_name}/weights/best.pt \
        --img-size 640 640

OR export to ONNX, with NMS included:

In [ ]:
!python export.py --weights runs/train/{model_name}/weights/best.pt --grid --end2end --simplify \
        --topk-all 100 --iou-thres 0.65 --conf-thres 0.35 --img-size 640 640 --max-wh 640

# Download compiled model weights

Set the value of `serialized_file_path` depending on whether you have exported to ONNX or TorchScript format

In [ ]:
weights_path = Path(f'runs/train/{model_name}/weights')
ts_path = weights_path/'best.torchscript.pt'
onnx_path = weights_path/'best.onnx'
serialized_file_path = ts_path.resolve()
#serialized_file_path = onnx_path.resolve()

kaggle_root = Path('/kaggle/working')
in_kaggle = kaggle_root.exists()
if in_kaggle:
    # In Kaggle, FileLink doesn't work if we don't cd to the root working directory
    # or if we pass an absolute path
    %cd /kaggle/working
    serialized_file_path = serialized_file_path.relative_to(kaggle_root)
FileLink(serialized_file_path)

# (Information) Generate TorchServe model archive file (.mar)

When we use TorchServe to serve our model, we first need to package it in a specific format (.mar). To generate a .mar file, we must use a different tool: **torch-model-archiver**.

You'd install torch-model-archiver with:
`pip install torch-model-archiver`

but we won't install it now as it's already included in the requirements of our server project.

Usage:
```
       torch-model-archiver [-h] --model-name MODEL_NAME
                            [--serialized-file SERIALIZED_FILE]
                            [--model-file MODEL_FILE] --handler HANDLER
                            [--extra-files EXTRA_FILES]
                            [--runtime {python,python2,python3}]
                            [--export-path EXPORT_PATH]
                            [--archive-format {tgz,no-archive,default}] [-f]
                            -v VERSION [-r REQUIREMENTS_FILE]
```

In this case, we will generate the model archive (.mar) file from a TorchScript or ONNX compiled model checkpoint. If we were instead passing an eager mode model through the parameter `serialized-file`, the parameter `model-file` would be required too.

Example:
```
serialized_file_path = f'runs/train/{model_name}/weights/best.torchscript.pt'
!torch-model-archiver --model-name cyclist_detector --version 1.0 --serialized-file $serialized_file_path \
--handler object_detector
```

We are not going to generate the .mar file from this notebook. It's is best done using the command line from our server project, because we need to use a custom handler. The default one for object detection (`--handler object_detector`) isn't adapted to our needs.

The script `<project_root>/server/objdetserver/scripts/generate_mar.py` chooses the proper handler for you.